# Downloading comments from youtube
In this section we will try different methods to download comments from youtube videos.

First we will try with conventional methods with requests and parsing the results with Beautiful Soup

In [14]:
import requests
import json
from bs4 import BeautifulSoup

USER_AGENT = 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/48.0.2564.116 Safari/537.36'

def find_value(html, key, num_chars=2):
    pos_begin = html.find(key) + len(key) + num_chars
    pos_end = html.find('"', pos_begin)
    return html[pos_begin: pos_end]

def ajax_request(session, url, params, data, retries=10, sleep=20):
    for _ in range(retries):
        response = session.post(url, params=params, data=data)
        if response.status_code == 200:
            response_dict = json.loads(response.text)
            return response_dict.get('page_token', None), response_dict['html_content']
        else:
            time.sleep(sleep)

def get_comments(youtube_id):
    youtube_comments_url = "https://www.youtube.com/all_comments?v=%s" % youtube_id
    youtube_comments_ajax_url = "https://www.youtube.com/comment_ajax"
    

In [15]:
youtube_id = "1Y-Au-tnBLs"
youtube_comments_url = "https://www.youtube.com/all_comments?v=%s" % youtube_id
youtube_comments_ajax_url = "https://www.youtube.com/comment_ajax"

session = requests.Session()
session.headers['User-Agent'] = USER_AGENT
response = session.get(youtube_comments_url)
soup = BeautifulSoup(response.content)

page_token = find_value(response.text, 'data-token')
xsrf_token = find_value(response.text, "XSRF_TOKEN", 4)

data = {'video_id': youtube_id,
                'session_token': xsrf_token}

params = {'action_load_comments': 1,
                  'order_by_time': True,
                  'filter': youtube_id}

first_iteration = True
if first_iteration:
    params['order_menu'] = True
else:
    data['page_token'] = page_token

response = ajax_request(session, youtube_comments_ajax_url, params, data)
print(response)
#print(page_token)
#print(xsrf_token)
#print(soup.prettify())

/usr/local/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 170 of the file /usr/local/Cellar/python3/3.5.1/Frameworks/Python.framework/Versions/3.5/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "html.parser")

  markup_type=markup_type))


('', '  \n\n      <div class="comment-entry">\n        \n\n\n\n\n\n\n\n\n  <div class="comment-item yt-commentbox-top" data-aid="UCVQrCq0eIS6OoYnJraGJVnw" data-cid="zXUFGiMTkrTUZommMK5zcQ5CJN_GkVVa5JIf_YRo7AI" data-vid="1Y-Au-tnBLs" data-name="smashingbear">\n\n\n    <a href="/user/smashingbear" target="_blank" class="g-hovercard" data-ytid="UCVQrCq0eIS6OoYnJraGJVnw">\n      <img class="user-photo" src="https://gp6.googleusercontent.com/-HDnFZ7YQYtc/AAAAAAAAAAI/AAAAAAAAAAA/GHpicuwqoG8/s48-c-k-no/photo.jpg?sz=50" width="48">\n    </a>\n    <div class="content">\n        <div class="comment-header">\n    <a href="/user/smashingbear" class="user-name g-hovercard" target="_blank" data-ytid="UCVQrCq0eIS6OoYnJraGJVnw">smashingbear</a>\n    <span class="spacer"></span>\n    <span class="time">\n        <a href="/watch?v=1Y-Au-tnBLs&amp;lc=zXUFGiMTkrTUZommMK5zcQ5CJN_GkVVa5JIf_YRo7AI" target="_blank">\n          5 years ago\n        </a>\n    </span>\n  </div>\n\n        <div class="comment-tex

In [5]:
import os
import sys
import time
import json
import requests
import argparse
import lxml.html

from lxml.cssselect import CSSSelector

YOUTUBE_COMMENTS_URL = 'https://www.youtube.com/all_comments?v={youtube_id}'
YOUTUBE_COMMENTS_AJAX_URL = 'https://www.youtube.com/comment_ajax'

USER_AGENT = 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/48.0.2564.116 Safari/537.36'


def find_value(html, key, num_chars=2):
    pos_begin = html.find(key) + len(key) + num_chars
    pos_end = html.find('"', pos_begin)
    return html[pos_begin: pos_end]


def extract_comments(html):
    tree = lxml.html.fromstring(html)
    item_sel = CSSSelector('.comment-item')
    text_sel = CSSSelector('.comment-text-content')
    time_sel = CSSSelector('.time')
    author_sel = CSSSelector('.user-name')

    for item in item_sel(tree):
        yield {'cid': item.get('data-cid'),
               'text': text_sel(item)[0].text_content(),
               'time': time_sel(item)[0].text_content().strip(),
               'author': author_sel(item)[0].text_content()}


def extract_reply_cids(html):
    tree = lxml.html.fromstring(html)
    sel = CSSSelector('.comment-replies-header > .load-comments')
    return [i.get('data-cid') for i in sel(tree)]


def ajax_request(session, url, params, data, retries=10, sleep=20):
    for _ in range(retries):
        response = session.post(url, params=params, data=data)
        if response.status_code == 200:
            response_dict = json.loads(response.text)
            return response_dict.get('page_token', None), response_dict['html_content']
        else:
            time.sleep(sleep)


def download_comments(youtube_id, sleep=1):
    session = requests.Session()
    session.headers['User-Agent'] = USER_AGENT

    # Get Youtube page with initial comments
    response = session.get(YOUTUBE_COMMENTS_URL.format(youtube_id=youtube_id))
    html = response.text
    reply_cids = extract_reply_cids(html)

    ret_cids = []
    for comment in extract_comments(html):
        ret_cids.append(comment['cid'])
        yield comment

    page_token = find_value(html, 'data-token')
    session_token = find_value(html, 'XSRF_TOKEN', 4)

    first_iteration = True

    # Get remaining comments (the same as pressing the 'Show more' button)
    while page_token:
        data = {'video_id': youtube_id,
                'session_token': session_token}

        params = {'action_load_comments': 1,
                  'order_by_time': True,
                  'filter': youtube_id}

        if first_iteration:
            params['order_menu'] = True
        else:
            data['page_token'] = page_token

        response = ajax_request(session, YOUTUBE_COMMENTS_AJAX_URL, params, data)
        if not response:
            break

        page_token, html = response

        reply_cids += extract_reply_cids(html)
        for comment in extract_comments(html):
            if comment['cid'] not in ret_cids:
                ret_cids.append(comment['cid'])
                yield comment

        first_iteration = False
        time.sleep(sleep)

    # Get replies (the same as pressing the 'View all X replies' link)
    for cid in reply_cids:
        data = {'comment_id': cid,
                'video_id': youtube_id,
                'can_reply': 1,
                'session_token': session_token}

        params = {'action_load_replies': 1,
                  'order_by_time': True,
                  'filter': youtube_id,
                  'tab': 'inbox'}

        response = ajax_request(session, YOUTUBE_COMMENTS_AJAX_URL, params, data)
        if not response:
            break

        _, html = response

        for comment in extract_comments(html):
            if comment['cid'] not in ret_cids:
                ret_cids.append(comment['cid'])
                yield comment
        time.sleep(sleep)

In [21]:
youtube_id = "1Y-Au-tnBLs"
for comment in download_comments(youtube_id):
    print(comment['text'])

heerlijke reclame...degenen die erop volgen, vind ik echt een heel stuk minder....dit is nog steeds de beste in de serie﻿
love this reclame, can't stop laughing :D﻿
0:26 I mean....HIHIHIHIHIHI :-) great job!﻿
hahahaha fucking grappigggg! LMAO﻿
The guy with the blue shirt on the right. THE GUY WITH THE BLUE SHIRT!! That is what I call Passion!﻿
Hahaha! GE-WEL-DIG!!

In 1 woord KONING!﻿
Geniaal! vooral dat gezicht op 0:25﻿
Julie denken niet na bij deze reclame.....﻿
jajaja la wuea.. :D quiero una pieza asi﻿
BEST COMMERCIAL﻿
deze is ehct vet man﻿
wiiiiiiiiiiiiiiiiiiiiiickeeeeeeeeeeeddddddddddd.......﻿
fcking Lau!!﻿
whahahaha﻿
hahahha xD 
cant stop laughin xD﻿
LOL XDDDDDDDD﻿
haha, fucking awesome advert!﻿
so fucking great!﻿
dit is mijn favoriet;p﻿
Briljant!﻿
echt geweldig is deze reclame:D:D:D﻿
omfg, f*ckin' pure awsomeness!!﻿
zajebista reklama﻿


# Reading the dataset

In [10]:
import csv
with open('./comedy_comparisons.test') as csvfile:
    youtube_id_reader = csv.reader(csvfile, delimiter = ',')
    i = 0
    for row in youtube_id_reader:
        if i > 3: break
        youtube_id = row[1]
        for comment in download_comments(youtube_id):
            try:
                print(comment['text'])
            except:
                pass
        i = i + 1
        print(str(i) + ": -" * 15 + "\n\n")

heerlijke reclame...degenen die erop volgen, vind ik echt een heel stuk minder....dit is nog steeds de beste in de serie﻿
love this reclame, can't stop laughing :D﻿
0:26 I mean....HIHIHIHIHIHI :-) great job!﻿
hahahaha fucking grappigggg! LMAO﻿
The guy with the blue shirt on the right. THE GUY WITH THE BLUE SHIRT!! That is what I call Passion!﻿
Hahaha! GE-WEL-DIG!!

In 1 woord KONING!﻿
Geniaal! vooral dat gezicht op 0:25﻿
Julie denken niet na bij deze reclame.....﻿
jajaja la wuea.. :D quiero una pieza asi﻿
BEST COMMERCIAL﻿
deze is ehct vet man﻿
wiiiiiiiiiiiiiiiiiiiiiickeeeeeeeeeeeddddddddddd.......﻿
fcking Lau!!﻿
whahahaha﻿
hahahha xD 
cant stop laughin xD﻿
LOL XDDDDDDDD﻿
haha, fucking awesome advert!﻿
so fucking great!﻿
dit is mijn favoriet;p﻿
Briljant!﻿
echt geweldig is deze reclame:D:D:D﻿
omfg, f*ckin' pure awsomeness!!﻿
zajebista reklama﻿
1: -: -: -: -: -: -: -: -: -: -: -: -: -: -: -


"Hilarious" isn't the word I would use. I would use "boring".﻿
great video !!!!!!!!!!﻿
that's so 

ParserError: Document is empty